In [1]:
from huggingface_hub import notebook_login

notebook_login()  # same as before

In [2]:
import os
from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset
from peft import AutoPeftModelForCausalLM, LoraConfig
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments

from trl import SFTTrainer
from trl.trainer import ConstantLengthDataset

In [3]:

def chars_token_ratio(dataset, tokenizer, nb_examples=400):
    """
    Estimate the average number of characters per token in the dataset.
    """
    total_characters, total_tokens = 0, 0
    for _, example in tqdm(zip(range(nb_examples), iter(dataset)), total=nb_examples):
        text = prepare_sample_text(example)
        total_characters += len(text)
        if tokenizer.is_fast:
            total_tokens += len(tokenizer(text).tokens())
        else:
            total_tokens += len(tokenizer.tokenize(text))

    return total_characters / total_tokens


def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


In [6]:
!pwd

/home/jovyan/sai


In [17]:
data = load_dataset('json',data_files='mungem.jsonl',split = "train",use_auth_token=True)
data = data.train_test_split(test_size=0.005, seed=None)
train_data = data["train"]
valid_data = data["test"]
print(f"Size of the train set: {len(train_data)}. Size of the validation set: {len(valid_data)}")

Size of the train set: 229. Size of the validation set: 2


In [4]:
def prepare_sample_text(example):
    """Prepare the text from a sample of the dataset."""
    text = f"{example['prompt']}\n {example['response']}"
    return text


def create_datasets(tokenizer):
    dataset = load_dataset('json',data_files='mungem.jsonl',split = "train",use_auth_token=True)
    dataset = dataset.train_test_split(test_size=0.005, seed=None)
    train_data = dataset["train"]
    valid_data = dataset["test"]
    print(f"Size of the train set: {len(train_data)}. Size of the validation set: {len(valid_data)}")

    chars_per_token = chars_token_ratio(train_data, tokenizer)
    print(f"The character to token ratio of the dataset is: {chars_per_token:.2f}")

    train_dataset = ConstantLengthDataset(
        tokenizer,
        train_data,
        formatting_func=prepare_sample_text,
        infinite=True,
        seq_length=256,
        chars_per_token=chars_per_token,
    )
    valid_dataset = ConstantLengthDataset(
        tokenizer,
        valid_data,
        formatting_func=prepare_sample_text,
        infinite=False,
        seq_length=256,
        chars_per_token=chars_per_token,
    )
    return train_dataset, valid_dataset

In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

base_model = AutoModelForCausalLM.from_pretrained(
    "PharMolix/BioMedGPT-LM-7B",
    quantization_config=bnb_config,
    device_map={"": 0},
    trust_remote_code=True,
    use_auth_token=True,
)
base_model.config.use_cache = False

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

/home/jovyan/.conda/envs/ppo/lib/python3.9/site-packages/transformers/models/auto/auto_factory.py:479: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/jovyan/.conda/envs/ppo/lib/python3.9/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/home/jovyan/.conda/envs/ppo/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/jovyan/.conda/envs/ppo/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was det

In [6]:
tokenizer = AutoTokenizer.from_pretrained("PharMolix/BioMedGPT-LM-7B", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training


training_args = TrainingArguments(
    output_dir="sft_biomedgpt7b",
    per_device_train_batch_size=10,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=10,
    learning_rate=2e-4,
    logging_steps=10,
    max_steps=105,
    report_to="tensorboard",
    save_steps=100,
    lr_scheduler_type="cosine",
    warmup_steps=1,
    optim="paged_adamw_32bit",
    fp16=True,
    remove_unused_columns=False,
    run_name="sft_biomedgpt7b",
)

train_dataset, eval_dataset = create_datasets(tokenizer)

/home/jovyan/.conda/envs/ppo/lib/python3.9/site-packages/datasets/load.py:2089: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=True' instead.
  warnings.warn(


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Size of the train set: 229. Size of the validation set: 2


 57%|█████▋    | 229/400 [00:00<00:00, 401.54it/s]

The character to token ratio of the dataset is: 2.59



/home/jovyan/.conda/envs/ppo/lib/python3.9/site-packages/trl/trainer/utils.py:465: UserWarning: The passed formatting_func has more than one argument. Usually that function should have a single argument `example` which corresponds to the dictionary returned by each element of the dataset. Make sure you know what you are doing.
  warnings.warn(


In [7]:
trainer = SFTTrainer(
    model=base_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    packing=True,
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_args,
)
trainer.train()
trainer.save_model("sft_biomedgpt7b")

output_dir = os.path.join("sft_biomedgpt7b", "final_checkpoint")
trainer.model.save_pretrained(output_dir)

# Free memory for merging weights
del base_model
torch.cuda.empty_cache()

model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map="auto", torch_dtype=torch.float16)
model = model.merge_and_unload()

output_merged_dir = os.path.join("sft_biomedgpt7b", "final_merged_checkpoint")
model.save_pretrained(output_merged_dir, safe_serialization=True)

/home/jovyan/.conda/envs/ppo/lib/python3.9/site-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
CUDA extension not installed.
CUDA extension not installed.
/home/jovyan/.conda/envs/ppo/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:166: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/home/jovyan/.conda/envs/ppo/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:227: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` metho

Step,Training Loss
10,2.572900
20,2.291400
30,2.087000
40,1.865200
50,1.779200
60,1.728700
70,1.704700
80,1.651800
90,1.620500
100,1.650300


/home/jovyan/.conda/envs/ppo/lib/python3.9/site-packages/trl/trainer/utils.py:487: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")
/home/jovyan/.conda/envs/ppo/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/jovyan/.conda/envs/ppo/lib/python3.9/site-packages/trl/trainer/utils.py:487: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/jovyan/.conda/envs/ppo/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/jovyan/.conda/envs/ppo/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/jovyan/.conda/envs/ppo/lib/python3.9